In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import math
import os

# Data Cleaning

In [2]:
# Loading all 3 databases

all_df = pd.read_csv('./data/dataframes/full_period.csv')
all_df = all_df.sort_values('year_quarter').reset_index(drop=True)

before_df = pd.read_csv('./data/dataframes/before_covid.csv')
before_df = before_df.sort_values('year_quarter').reset_index(drop=True)

during_df = pd.read_csv('./data/dataframes/during_covid.csv')
during_df = during_df.sort_values('year_quarter').reset_index(drop=True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,11,13,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
all_df.shape

(2561175, 17)

In [4]:
before_df.shape

(1686178, 17)

In [5]:
during_df.shape

(874997, 17)

In [6]:
# Define data cleaning function

def clean_data(df):
    all_df = df

    # drop turnover missing values
    all_df = all_df[all_df['Turnover'].notna()]

    # Remove all business types except äriühing
    all_df = all_df[all_df['Type']=="Äriühing"]

    # Drop duplicates
    all_df = all_df.drop_duplicates()

    # impute missing values
    all_df['County'] = all_df['County'].replace(np.nan, "other")
    all_df['Business_area_by_EMTAK_system'] = all_df['Business_area_by_EMTAK_system'].fillna("other")

    # same as number of employee, will do it by grouping during feat eng
    #all_df['All_taxes'] = all_df['All_taxes'].fillna(all_df['All_taxes'].mean())
    #all_df['Labor_taxes'] = all_df['Labor_taxes'].fillna(all_df['Labor_taxes'].mean())

    # will add this to the feat engineering
    # to do it by grouping
    #all_df['number_of_employees'] = all_df['number_of_employees'].fillna(int(all_df['number_of_employees'].mean()))

    # nan values mean companu still exist
    all_df['Deletion_Date']=all_df['Deletion_Date'].fillna(int(1))
    # Other values mean company died
    all_df.loc[(all_df['Deletion_Date'] !=1), 'Deletion_Date'] = 0

    #replacing negative turover with small nr
    all_df['Turnover'] = all_df['Turnover'].apply(lambda x : x if x > 0 else 0.000001)

    #dropping name_y
    #all_df.drop(columns=['Name_y'], inplace=True)

    return all_df

In [7]:
# Cleaning datasets

all_cl = clean_data(all_df).reset_index(drop=True)
before_cl = clean_data(before_df).reset_index(drop=True)
during_cl = clean_data(during_df).reset_index(drop=True)
#all_cl

In [8]:
all_cl.shape

(1675254, 17)

In [9]:
before_cl.shape

(1101055, 17)

In [10]:
during_cl.shape

(574199, 17)

In [11]:
# Testing on company Nortal AS
print('Whole period')
print(all_cl.query('Registration_number == 10391131').shape)
print(all_df.query('Registration_number == 10391131').shape)

print('Before covid')
print(before_cl.query('Registration_number == 10391131').shape)
print(before_df.query('Registration_number == 10391131').shape)

print('During covid')
print(during_cl.query('Registration_number == 10391131').shape)
print(during_df.query('Registration_number == 10391131').shape)

Whole period
(19, 17)
(19, 17)
Before covid
(13, 17)
(13, 17)
During covid
(6, 17)
(6, 17)


# Feature Engineering

In [12]:
# Defining engineering engine

def feat_eng(g):
    ex_df=pd.DataFrame(g)

    if ex_df['number_of_employees'].isnull().sum()>0:

        try:
            ex_df['number_of_employees'] = ex_df['number_of_employees'].fillna(int(ex_df['number_of_employees'].mean()))
        except:
            ex_df['number_of_employees'] = ex_df['number_of_employees'].fillna(int(1))



    if ex_df['Labor_taxes'].isnull().sum():
        #print("Labor_taxes")
        if math.isnan(ex_df['Labor_taxes'].mean()):
            ex_df['Labor_taxes'] = 0000.1
        else:
            ex_df['Labor_taxes'] = ex_df['Labor_taxes'].fillna(ex_df['Labor_taxes'].mean())


    if ex_df['All_taxes'].isnull().sum():
        if math.isnan(ex_df['All_taxes'].mean()):
            ex_df['All_taxes'] = 0000.1
        else:
            ex_df['All_taxes'] = ex_df['All_taxes'].fillna(ex_df['All_taxes'].mean())



    # Cumulative Turn over
    ex_df['cum_sum_turnover'] = np.round(ex_df['Turnover'].cumsum(), 2)
    ex_df['cum_perc_turnover'] = np.round(100*ex_df['cum_sum_turnover']/ex_df['Turnover'].sum(), 2)

    # combined taxes
    ex_df['combined_taxes'] = np.round(ex_df['Labor_taxes'] + ex_df['All_taxes'], 2)

    # Cumulative Taxes
    ex_df['cum_sum_combined_taxes'] = np.round(ex_df['combined_taxes'].cumsum(), 2)
    #ex_df['cum_perc_combined_taxes'] = np.round(100*ex_df['cum_sum_combined_taxes']/ex_df['combined_taxes'].sum(), 2)

    # Revenue Growth
    ex_df['revenue_growth'] = np.round(ex_df.Turnover.pct_change(fill_method="ffill"),2)
    ex_df['revenue_growth'] = ex_df['revenue_growth'].fillna(0)


    # Profit per Employee
    ex_df["profit_per_employee"] = np.round(ex_df['Turnover']/ex_df['number_of_employees'], 2)


    return ex_df

In [13]:
# Doing magic on whole period data

all_cl = all_cl.groupby("Registration_number").apply(feat_eng)
all_cl

,Registration_number,Name,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Institutional_classification,Client_ID_by_Eesti_Pank,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee
0,10257616,"PEX TALLA, OÜ",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,24830.48,15559.13,149408.76,8.0,...,S11002-S11003,8.0,1997-09-01,1,149408.76,7.06,40389.61,40389.61,0.00,18676.10
1,12994847,MAD & WONDERFUL OÜ,Äriühing,jah,Tartu maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,5.87,0.00,163.51,1.0,...,S11002-S11003,8.0,2016-02-12,1,163.51,0.06,5.87,5.87,0.00,163.51
2,11152753,DZORDZIA OÜ,Äriühing,jah,Harju maakond,VEONDUS JA LAONDUS,6534.79,3163.41,116510.14,5.0,...,S11002-S11003,8.0,2005-07-14,1,116510.14,8.25,9698.20,9698.20,0.00,23302.03
3,12994862,O KAKS SERVICE OÜ,Äriühing,jah,Harju maakond,EHITUS,2821.08,899.37,21054.69,5.0,...,S11002-S11003,8.0,2016-02-12,1,21054.69,1.08,3720.45,3720.45,0.00,4210.94
4,12994879,TERKEN OÜ,Äriühing,jah,Lääne maakond,MAJUTUS JA TOITLUSTUS,1324.33,613.10,6251.00,1.0,...,S11002-S11003,8.0,2016-02-12,1,6251.00,5.81,1937.43,1937.43,0.00,6251.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675249,14798832,COLART GROUP OÜ,Äriühing,jah,Harju ( Tallinn ),EHITUS,4501.30,3352.39,20663.49,2.0,...,S11002-S11003,8.0,2019-09-06,1,380420.72,100.00,7853.69,65976.16,-0.57,10331.75
1675250,10942467,"TONNINE, OÜ",Äriühing,jah,Tartu ( Tartu linn ),KINNISVARAALANE TEGEVUS,543.53,223.74,1039.75,1.0,...,S11002-S11003,8.0,2003-05-05,1,81608.72,100.00,767.27,35184.67,0.98,1039.75
1675251,14798869,WISEDRIVE OÜ,Äriühing,jah,Harju ( Kiili vald ),INFO JA SIDE,0.00,3699.91,5535.42,2.0,...,S11002-S11003,8.0,2019-09-06,1,14506.41,100.00,3699.91,16847.43,1.64,2767.71
1675252,10942507,"TARVO AUTO, OÜ",Äriühing,jah,Valga ( Valga vald ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,8720.04,6099.77,47566.62,5.0,...,S11002-S11003,8.0,2003-05-06,1,772702.23,100.00,14819.81,245645.25,-0.13,9513.32


In [14]:
# Saving cleared and engineered data

all_cl.to_csv('./data/dataframes/full_period_enginered.csv', index=False)
# Just to add something

In [15]:
all_cl.query('Registration_number == 10391131')

,Registration_number,Name,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Institutional_classification,Client_ID_by_Eesti_Pank,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee
42705,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1426220.77,994053.02,8609633.24,265.0,...,S11002-S11003,8.0,1998-02-20,1,8.609633e+06,4.82,2420273.79,2420273.79,0.00,32489.18
154027,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,2147796.23,1648346.87,5743574.41,297.0,...,S11002-S11003,8.0,1998-02-20,1,1.435321e+07,8.04,3796143.10,6216416.89,-0.33,19338.63
160950,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1550912.46,1195574.24,4944107.48,290.0,...,S11002-S11003,8.0,1998-02-20,1,1.929732e+07,10.81,2746486.70,8962903.59,-0.14,17048.65
320600,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1670639.48,1158832.55,7469944.39,287.0,...,S11002-S11003,8.0,1998-02-20,1,2.676726e+07,14.99,2829472.03,11792375.62,0.51,26027.68
349819,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1821672.37,1216835.81,13929933.98,286.0,...,S11002-S11003,8.0,1998-02-20,1,4.069719e+07,22.80,3038508.18,14830883.80,0.86,48706.06
415019,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1650326.66,1280771.24,5916328.85,334.0,...,S11002-S11003,8.0,1998-02-20,1,4.661352e+07,26.11,2931097.90,17761981.70,-0.58,17713.56
554640,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1789294.84,1403898.72,9355564.24,333.0,...,S11002-S11003,8.0,1998-02-20,1,5.596909e+07,31.35,3193193.56,20955175.26,0.58,28094.79
628381,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,1812415.29,1446988.51,5548868.31,341.0,...,S11002-S11003,8.0,1998-02-20,1,6.151795e+07,34.46,3259403.80,24214579.06,-0.41,16272.34
704249,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,2660268.53,1569078.73,14522027.70,346.0,...,S11002-S11003,8.0,1998-02-20,1,7.603998e+07,42.59,4229347.26,28443926.32,1.62,41971.18
788907,10391131,NORTAL AS,Äriühing,jah,Harju maakond,INFO JA SIDE,2104782.75,1801131.72,7352014.55,360.0,...,S11002-S11003,8.0,1998-02-20,1,8.339200e+07,46.71,3905914.47,32349840.79,-0.49,20422.26


In [16]:
# Not necessary as will be divided into parts after cleaning/engineering/labelling is complete
# ... with pre-covid period of data
'''
before_cl = before_cl.groupby("Registration_number").apply(feat_eng)
before_cl

SyntaxError: EOF while scanning triple-quoted string literal (2396848970.py, line 6)

In [ ]:
# ... and with the period of pandemic

during_cl = during_cl.groupby("Registration_number").apply(feat_eng)
during_cl


In [ ]:
before_cl.to_csv('./data/dataframes/before_covid_enginered.csv', index=False)

In [ ]:
during_cl.to_csv('./data/dataframes/during_covid_enginered.csv', index=False)
'''